In [1]:
import pandas as pd
import powerplantmatching as pm
import pypsa
import pycountry
import os 

In [2]:
def prepare_capacity_table(capacity):
    
    capacity_matching = pd.Series(
        ["nuclear", "lignite", "coal", "gas", "oil", "other", "ROR", "ROR", "reservoir", "reservoir", "PHS",
         "reservoir (pumping)", "PHS (pumping)", "onwind", "offwind", "CSP", "solar", "biomass", "biomass", "battery"],
        capacity.index
    )

    capacity = capacity.groupby(capacity_matching, ).sum()
    
    pm_plants = pm.powerplants()
    gas_share = pd.DataFrame()
    gas_share["CCGT"] = pm_plants.groupby(["Fueltype", "Technology", "Country"]).sum(numeric_only=True).loc["Natural Gas", "CCGT",:].Capacity.div(
        pm_plants.groupby(["Fueltype", "Technology", "Country"]).sum(numeric_only=True).loc["Natural Gas", :].Capacity.groupby(level=1).sum(),
        fill_value=0
    )

    gas_share.index = [pycountry.countries.lookup(i).alpha_2 for i in gas_share.index]

    gas_share["OCGT"] = 1- gas_share["CCGT"]

    gas_share = gas_share.reindex(capacity.columns.str[:2]).fillna(gas_share.mean())
    gas_share.index = capacity.columns

    capacity = pd.concat([
        capacity.drop("gas"),
        gas_share.multiply(capacity.loc["gas"],axis=0).T
    ])
    
    return capacity


In [3]:
def build_capacity_table():

    capacity_years = pd.DataFrame()

    for sheet in plant_sheets.index:

        capacity_raw = pd.read_excel(excel_file, sheet, header=1, index_col=1).dropna(how="all", axis=1).iloc[:20]

        capacity_years[sheet[2:]] = prepare_capacity_table(capacity_raw).stack().astype(float)

    capacity_years.columns = capacity_years.columns.astype(int)
    capacity_years = capacity_years.reindex(years,axis=1)
    capacity_years.loc[distributed_resources, :] = capacity_years.loc[distributed_resources, :].interpolate(axis=1)
    

    return capacity_years.fillna(method="ffill", axis=1)[year].unstack(1)

In [4]:
def build_thermal_properties(properties):
    
    for col in properties.columns:
        try: 
            properties[col] = properties[col].astype(float)
        except:
            None

    properties.set_index(properties.index.remove_unused_levels(), inplace=True)

    properties_matching = pd.Series(
        ["OCGT", "coal", "oil", "oil", "lignite", "nuclear", "oil"],
        properties.index.levels[0]
    )

    properties_matching = properties_matching.reindex([i[0] for i in properties.index])

    properties_matching.index = properties.index

    properties_matching.loc[[i for i in properties_matching.index if "CCGT" in i[1]]] = "CCGT"

    properties_transformed = properties.iloc[:, 1:].groupby(
        properties_matching.reindex(properties.index).values
    ).mean()

    missing_properties = properties.reindex([("Gas", "conventional old 2"), ("Gas", "conventional old 2")]).copy()
    missing_properties.index = ["biomass", "other"]
    missing_properties.loc["biomass","CO2 emission factor"] = 0

    properties_transformed = pd.concat([properties_transformed, missing_properties[properties_transformed.columns]])

    properties_transformed.set_index(pd.Series("existing", properties_transformed.index),append=True, inplace=True)

    properties_new = properties.loc["Gas", ["CCGT new", "OCGT new"], :].set_index(pd.MultiIndex.from_product([["CCGT", "OCGT"], ["new"]]))

    properties_transformed = pd.concat([properties_transformed, properties_new[properties_transformed.columns]])
    properties_transformed.index.names=["carrier", "invest_status"]

    return properties_transformed

In [5]:
def build_inflows(inflows):
    base_year_hydro = pd.Series(
        inflows.index.levels[4].astype(int), 
        inflows.index.levels[4]).subtract(year).abs().idxmin()

    inflows = inflows.loc[:, str(climate_year), :, :, base_year_hydro]

    inflow_grouper = pd.Series(["ROR", "PHS", "reservoir", "reservoir", "ROR"], inflows.index.levels[1])

    inflows = inflows.unstack(1).groupby(inflow_grouper, axis=1).sum().multiply(1e3) # conversion to MWh
    
    inflows = inflows.unstack(0).stack(0)
    inflows.index = [" ".join(i) for i in inflows.index]
    
    inflows = inflows.T
    
    inflows.index = snapshots
    
    return inflows

In [6]:
def add_existing_storage():  
 
    storage = capacity.loc[["PHS", "reservoir", "battery"]].unstack().reset_index().copy()
    storage.columns = ["bus", "carrier", "p_nom"]
    storage.index = storage.bus + " " + storage.carrier

    p_min_pu = capacity.loc[["PHS (pumping)", "reservoir (pumping)",]].div(capacity.loc[["PHS", "reservoir"]].add(1e-5).values)
    p_min_pu.index = ["PHS", "reservoir"]

    storage["p_min_pu"] = (
        p_min_pu.unstack()
        .reindex(pd.MultiIndex.from_arrays([storage.bus, storage.carrier]))
    ).values

    storage.loc[storage.carrier=="battery", "p_min_pu"] = -1

    storage = storage[storage.p_nom >0]

    storage_inflows = inflows[storage.loc[storage.carrier=="reservoir"].index]

    storage_capacity_raw = pd.read_excel(excel_file, sheet, index_col=1, header=28).dropna(how="all", axis=1)

    storage_capacity= storage_capacity_raw.groupby(
        ["ROR", "reservoir", "reservoir", "PHS", "battery"]
    ).sum()

    storage["max_hours"] = storage_capacity.unstack().reindex(
        pd.MultiIndex.from_arrays(
            [storage.bus, storage.carrier]  
        )
    ).div(
        storage.p_nom.values
    ).values

    n.madd(
        "StorageUnit",
        storage.index,
        **storage,
        inflow=storage_inflows,
        invest_status = "existing"
    )

In [7]:
def prepare_commodity_prices(commodity_prices):

    commodity_prices = commodity_prices_raw.iloc[:, 2:].groupby(commodity_prices.index).mean()
    commodity_prices = commodity_prices.groupby(["co2", "CCGT", "coal", "oil", "hydrogen", "oil", "lignite", "nuclear", "oil"]).mean()
       
    converter = pd.DataFrame(3.6 , commodity_prices.index, commodity_prices.columns)
    converter["CO2"] = 3.6/1000
    converter.loc["co2"] = 1
    
    commodity_prices = converter*commodity_prices

    to_add = commodity_prices.reindex(["CCGT", "CCGT", "CCGT"])
    to_add.index = ["OCGT", "biomass", "other"]
    to_add.loc[["other"]] = 0
    to_add.loc[["biomass"]] = biomass_price
    to_add.loc[["biomass"], "CO2"] = 0
    commodity_prices = pd.concat([commodity_prices, to_add])
    
    return commodity_prices

In [8]:
def add_existing_dispatchables():
    
    dispatchable_existing = capacity.loc[dispatchable].unstack().reset_index()
    dispatchable_existing.columns = ["bus", "carrier", "p_nom"]
    dispatchable_existing.index = dispatchable_existing.bus + " " + dispatchable_existing.carrier
    dispatchable_existing = dispatchable_existing.loc[dispatchable_existing.p_nom >0]

    dispatchable_existing["efficiency"] = properties.loc[:, "existing", :].reindex(dispatchable_existing.carrier)["Standard efficiency in NCV terms"].values

    dispatchable_existing["start_up_cost"] = properties["Start-up fix cost (e.g. wear) warm start"].loc[:, "existing"].reindex(dispatchable_existing.carrier, level=1).fillna(0).values
    dispatchable_existing["ramp_limit_up"] = properties["Ramp up rate % of max output power / min"].loc[:, "existing"].reindex(dispatchable_existing.carrier).values*60
    dispatchable_existing["ramp_limit_down"] = properties["Ramp down rate % of max output power / min"].loc[:, "existing"].reindex(dispatchable_existing.carrier).values*60
    dispatchable_existing["p_min_pu"] = properties["Minimum stable generation (% of max power)"].loc[:, "existing"].reindex(dispatchable_existing.carrier).values
    dispatchable_existing["min_up_time"] = properties["Min Time on"].loc[:, "existing"].reindex(dispatchable_existing.carrier).values
    dispatchable_existing["min_down_time"] = properties["Min Time off"].loc[:, "existing"].reindex(dispatchable_existing.carrier).values
    
    
    
    dispatchable_existing["marginal_cost"] = commodity_prices.reindex(dispatchable_existing.carrier)[base_year].div(
        dispatchable_existing.efficiency.values,
    ).add(
        (
            commodity_prices.reindex(
                dispatchable_existing.carrier)["CO2"]
            .multiply(
                commodity_prices.loc["co2", base_year]
            )
        )
    ).add(
        properties.loc[:, "existing",:].reindex(dispatchable_existing.carrier)["Variable O&M cost"]
    ).values

    n.madd(
        "Generator",
        dispatchable_existing.index,
        **dispatchable_existing,
        invest_status = "existing"
    )

In [9]:
def add_renewables():
    
    res = capacity.loc[["onwind","offwind", "solar", "CSP", "ROR"]].unstack().copy()
    res = res.reset_index()
    res.columns = ["bus", "carrier", "p_nom"]
    res.index = res.bus + " " + res.carrier

    res = res[res.p_nom>0]

    vre = ["onwind", "offwind", "solar", "CSP"]

    base_year_res = int(plant_sheets[plant_sheets >= year].subtract(year).idxmin()[2:])
    
    p_max_pu = pd.concat(
        [pd.read_hdf("resources/res_profile.h5", tech).loc[:, str(climate_year), str(base_year_res), :] for tech in vre],
        axis=1
    )

    p_max_pu.columns = vre
    p_max_pu = p_max_pu.unstack(0).stack(0)
    p_max_pu.index = [" ".join(i) for i in p_max_pu.index]
    p_max_pu.columns = snapshots
    p_max_pu = p_max_pu.T

    p_max_pu = pd.concat(
        [p_max_pu, inflows[res.filter(like="ROR", axis=0).index].div(res.filter(like="ROR", axis=0).p_nom)],
        axis=1
    )

    n.madd(
        "Generator",
        res.index,
        **res,
        p_max_pu = p_max_pu[res.index],
        invest_status="policy"
    )

In [10]:
def group_luxembourg(demand, links):
    
    demand_grouper = pd.Series(demand.columns, demand.columns)
    demand_grouper.loc[demand_grouper.index.str[:2] == "LU"] = "LUG1"
    demand = demand.groupby(demand_grouper, axis=1).sum()

    links.loc[links.bus0.str[:2] == "LU", "bus0"] = "LUG1"
    links.loc[links.bus1.str[:2] == "LU", "bus1"] = "LUG1"
    
    return demand, links

In [11]:
def add_dsr():
    
    p_nom_dsr.columns = ["band " + i.split(" ")[2] for i in p_nom_dsr.columns]

    marginal_cost_dsr.columns = ["band " + i.split(" ")[8] for i in marginal_cost_dsr.columns]

    dsr = pd.concat([p_nom_dsr.stack(), marginal_cost_dsr.stack()],axis=1)

    dsr = dsr.loc[:, base_year, :]

    dsr = dsr.set_index((i[0] + " dsr " + i[1] for i in dsr.index), append=True).reset_index([0,1]).drop("level_1",axis=1)

    dsr.columns = ["bus", "p_nom", "marginal_cost"]
    
    dsr["carrier"] = "dsr"

    n.madd(
        "Generator",
        dsr.index,
        **dsr,
        invest_status = "existing"
    )

In [12]:
def add_dispatchable_investment_options():

    zones_for_investment = capacity.sum()[capacity.loc[dispatchable].sum() >0 ].index # no investment in offshores zones etc.

    new_dispatchables = pd.DataFrame(
        0.01, 
        index= pd.MultiIndex.from_product(
            [zones_for_investment,technologies_for_investment], 
            names=["bus", "carrier"]), 
        columns=["p_nom"]
    )

    new_dispatchables.reset_index(inplace=True)

    new_dispatchables.index = new_dispatchables.bus + " " + new_dispatchables.carrier + " new"

    new_dispatchables["efficiency"] = properties.loc[:, "new", :]["Standard efficiency in NCV terms"].reindex(new_dispatchables.carrier).values

    new_dispatchables["start_up_cost"] = properties["Start-up fix cost (e.g. wear) warm start"].loc[:, "new"].reindex(new_dispatchables.carrier, level=1).fillna(0).values

    new_dispatchables["ramp_limit_up"] = properties["Ramp up rate % of max output power / min"].loc[:, "new"].reindex(new_dispatchables.carrier).values*60
    new_dispatchables["ramp_limit_down"] = properties["Ramp down rate % of max output power / min"].loc[:, "new"].reindex(new_dispatchables.carrier).values*60
    new_dispatchables["p_min_pu"] = properties["Minimum stable generation (% of max power)"].loc[:, "new"].reindex(new_dispatchables.carrier).values
    new_dispatchables["min_up_time"] = properties["Min Time on"].loc[:, "new"].reindex(new_dispatchables.carrier).values
    new_dispatchables["min_down_time"] = properties["Min Time off"].loc[:, "new"].reindex(new_dispatchables.carrier).values

    new_dispatchables["marginal_cost"] = commodity_prices.reindex(new_dispatchables.carrier)[base_year].div(
            new_dispatchables.efficiency.values,
        ).add(
            (
                commodity_prices.reindex(
                    new_dispatchables.carrier)["CO2"]
                .multiply(
                    commodity_prices.loc["co2", base_year]
                )
            )
        ).add(
            properties.loc[:, "existing",:].reindex(new_dispatchables.carrier)["Variable O&M cost"]
        ).values

    n.madd(
        "Generator",
        new_dispatchables.index,
        **new_dispatchables,
        invest_status = "new"
    )

In [13]:
def set_investment_bounds():
    
    n.generators.loc[n.generators.invest_status == "existing", "p_nom_max"] = n.generators.loc[n.generators.invest_status == "existing", "p_nom"].clip(0.01)
    n.generators.loc[n.generators.invest_status == "existing", "p_nom"] = n.generators.loc[n.generators.invest_status == "existing", "p_nom"].clip(0.01)
    n.generators.p_nom_min = 0.01
    n.generators.loc[n.generators.invest_status == "policy", "p_nom_max"] = n.generators.loc[n.generators.invest_status == "policy", "p_nom"]
    n.generators.loc[n.generators.invest_status == "policy", "p_nom_min"] = n.generators.loc[n.generators.invest_status == "policy", "p_nom"]

In [14]:
commodity_prices_raw = pd.read_excel("data/Fuel and CO2 prices_ERAA2023.xlsx", index_col = 0)

In [15]:
year = 2029

In [16]:
climate_year = 1990

In [17]:
years = range(2025, 2034)

In [18]:
save_path = "resources/networks/base/cy" + str(climate_year) + "_ty" + str(year) + ".nc"

In [19]:
biomass_price = 7 # make adjustable in config

In [20]:
dispatchable = ["CCGT", "OCGT", 'biomass', 'coal', 'lignite', 'nuclear','oil',  'other', ]

In [21]:
technologies_for_investment = ["OCGT", "CCGT"]

In [22]:
inflows_raw = pd.read_hdf("resources/inflow.h5", "inflow")

In [23]:
excel_file = pd.ExcelFile("data/pemmdb.xlsx")

In [24]:
snapshots = pd.date_range(start="2010-01-01", freq="h", periods=8760)

In [25]:
plant_sheets = [i for i in excel_file.sheet_names if "TY" in i]
plant_sheets = pd.Series([int(i[2:]) for i in plant_sheets], plant_sheets )

In [26]:
sheet = plant_sheets[plant_sheets <= year].subtract(year).idxmax()

In [27]:
base_year = int(sheet[2:])

In [28]:
distributed_resources = ["onwind", "offwind", "CSP","solar", "battery"]

In [29]:
demand = pd.read_hdf("resources/demand.h5")
demand = demand.loc[:, climate_year, :, str(base_year), :].unstack(1)
demand.index = snapshots
demand.drop("TR00",axis=1, inplace=True)

In [30]:
links = pd.read_hdf("resources/ntcs.h5", "p_nom")
links_p_max_pu = pd.read_hdf("resources/ntcs.h5", "p_max_pu")
links = links[base_year].unstack(1)
links_p_max_pu = links_p_max_pu[base_year].unstack(2)
links_p_max_pu.index = snapshots

In [31]:
links.dropna(inplace=True)
links_p_max_pu.dropna(axis=1, inplace=True)

In [32]:
links["carrier"] = [i[-2:] for i in links.index]

In [33]:
demand, links = group_luxembourg(demand, links)

In [34]:
commodity_prices = prepare_commodity_prices(commodity_prices_raw)

In [35]:
properties_raw = pd.read_excel(excel_file, "Thermal Properties", index_col=[2,3], header=3).dropna(how="all").iloc[1:, 2:].dropna(how="all", axis=1).iloc[:24, :12]
properties_raw2 = pd.read_excel(excel_file, "Thermal Properties", index_col=[2,3], skiprows=35, header=[0,3]).iloc[:, 2:].dropna(how="all", axis=1).dropna(how="all")
properties_raw2.columns = [" ".join(i) for i in properties_raw2.columns]
properties_raw = pd.concat([properties_raw, properties_raw2],axis=1)

In [36]:
properties = build_thermal_properties(properties_raw)

In [37]:
n = pypsa.Network()
n.set_snapshots(snapshots)

In [38]:
capacity = build_capacity_table()

In [39]:
p_nom_dsr = pd.read_excel(excel_file, "Explicit DSR", index_col = [0,1]).iloc[:, :8]
marginal_cost_dsr = pd.read_excel(excel_file, "Explicit DSR", index_col = [0,1]).iloc[:, 8:16]

In [41]:
capacity

,AL00,AT00,BA00,BE00,BEOF,BG00,CH00,CY00,CZ00,DE00,...,RO00,RS00,SE01,SE02,SE03,SE04,SI00,SK00,UK00,UKNI
CSP,0.00000,0.000000,0.0,0.000000,0.0,0.000,0.000000,50.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
PHS,0.00000,450.000000,0.0,1305.000000,0.0,864.000,1900.000000,0.000000,688.050000,7009.840000,...,0.000000,614.000000,0.000000,0.000000,0.000000,0.000000,180.000000,760.000000,2744.000000,0.000000
PHS (pumping),0.00000,-450.000000,0.0,-1226.760000,0.0,-784.000,-1900.000000,0.000000,-664.000000,-7166.600000,...,0.000000,-560.000000,0.000000,0.000000,0.000000,0.000000,-185.000000,-690.000000,-2684.000000,0.000000
ROR,503.67200,6072.032760,149.4,142.602000,0.0,535.510,4215.000000,0.000000,431.630000,3933.900000,...,3337.544000,2123.820000,0.000000,0.000000,0.000000,0.000000,1135.374437,1626.192100,2183.139585,0.000000
battery,0.00000,59.656670,0.0,499.054400,0.0,1600.000,990.485950,0.000000,565.250000,2992.155000,...,365.000000,0.000000,8.405660,8.271634,120.194899,52.426062,546.800000,218.000000,13632.464500,266.000000
biomass,0.00000,702.096370,0.0,969.025000,0.0,282.000,397.942938,12.100000,707.500000,12056.636000,...,129.997800,130.990000,198.700000,743.570000,2818.055000,738.090000,58.859512,541.950408,2249.311272,82.000000
coal,0.00000,0.000000,0.0,0.000000,0.0,0.000,0.000000,0.000000,45.000000,5106.110000,...,130.000000,0.000000,0.000000,0.000000,0.000000,0.000000,81.000000,268.925712,0.000000,0.000000
lignite,0.00000,0.000000,1748.0,0.000000,0.0,2842.000,0.000000,0.000000,4971.800000,14222.000000,...,0.000000,4999.748000,0.000000,0.000000,0.000000,0.000000,539.000000,60.797169,0.000000,0.000000
nuclear,0.00000,0.000000,0.0,2077.000000,0.0,2000.000,2930.000000,0.000000,4047.200000,0.000000,...,650.000000,0.000000,0.000000,0.000000,6883.000000,0.000000,696.000000,2775.637264,4570.000000,0.000000
offwind,0.00000,0.000000,0.0,3310.000000,700.0,0.000,0.000000,0.000000,0.000000,22685.000000,...,500.000000,0.000000,0.000000,0.000000,715.000000,2315.071429,0.000000,0.000000,44132.469980,50.000000


In [40]:
buses = (
    capacity.sum()[capacity.sum() >0].index
    .union(demand.columns)
    .union(links.bus0.unique())
    .union(links.bus1.unique())
)

In [41]:
n.madd(
    "Bus", 
    buses, 
    carrier = "electricity", 
    country = buses.str[:2]
)

Index(['AL00', 'AT00', 'BA00', 'BE00', 'BEOF', 'BG00', 'CH00', 'CY00', 'CZ00',
       'DE00', 'DEKF', 'DKBH', 'DKE1', 'DKKF', 'DKW1', 'EE00', 'ES00', 'FI00',
       'FR00', 'GR00', 'GR03', 'HR00', 'HU00', 'IE00', 'ITCA', 'ITCN', 'ITCS',
       'ITN1', 'ITS1', 'ITSA', 'ITSI', 'ITVI', 'LT00', 'LUG1', 'LV00', 'ME00',
       'MK00', 'MT00', 'NL00', 'NLLL', 'NOM1', 'NON1', 'NOS0', 'PL00', 'PLE0',
       'PLI0', 'PT00', 'RO00', 'RS00', 'SE01', 'SE02', 'SE03', 'SE04', 'SI00',
       'SK00', 'UK00', 'UKNI'],
      dtype='object')

In [42]:
n.madd(
    "Load", 
    demand.columns,
    bus=demand.columns,
    p_set = demand
)

Index(['AL00', 'AT00', 'BA00', 'BE00', 'BG00', 'CH00', 'CY00', 'CZ00', 'DE00',
       'DKE1', 'DKW1', 'EE00', 'ES00', 'FI00', 'FR00', 'GR00', 'GR03', 'HR00',
       'HU00', 'IE00', 'ITCA', 'ITCN', 'ITCS', 'ITN1', 'ITS1', 'ITSA', 'ITSI',
       'LT00', 'LUG1', 'LV00', 'ME00', 'MK00', 'MT00', 'NL00', 'NOM1', 'NON1',
       'NOS0', 'PL00', 'PT00', 'RO00', 'RS00', 'SE01', 'SE02', 'SE03', 'SE04',
       'SI00', 'SK00', 'UK00', 'UKNI'],
      dtype='object', name='zone')

In [43]:
inflows = build_inflows(inflows_raw)

In [44]:
add_existing_storage()

In [45]:
add_existing_dispatchables()

In [46]:
add_renewables()

In [47]:
add_dsr()

In [48]:
add_dispatchable_investment_options()

In [49]:
set_investment_bounds()

In [50]:
links

,bus0,bus1,p_nom,carrier
AL00-GR00-AC,AL00,GR00,400,AC
AL00-ME00-AC,AL00,ME00,400,AC
AL00-MK00-AC,AL00,MK00,500,AC
AL00-RS00-AC,AL00,RS00,240,AC
AT00-CH00-AC,AT00,CH00,1200,AC
...,...,...,...,...
UK00-NL00-DC,UK00,NL00,1000,DC
UK00-NOS0-DC,UK00,NOS0,1464.000005,DC
UK00-UKNI-DC,UK00,UKNI,450,DC
UKNI-IE00-AC,UKNI,IE00,900,AC


In [52]:
n.madd(
    "Link",
    links.index,
    bus0 = links.bus0,
    bus1 = links.bus1,
    p_nom = links.p_nom,
    p_max_pu = links_p_max_pu,
    carrier = links.carrier
)

Index(['AL00-GR00-AC', 'AL00-ME00-AC', 'AL00-MK00-AC', 'AL00-RS00-AC',
       'AT00-CH00-AC', 'AT00-CZ00-AC', 'AT00-DE00-AC', 'AT00-HU00-AC',
       'AT00-ITN1-AC', 'AT00-SI00-AC',
       ...
       'UK00-BE00-DC', 'UK00-DE00-DC', 'UK00-DKW1-DC', 'UK00-FR00-DC',
       'UK00-IE00-DC', 'UK00-NL00-DC', 'UK00-NOS0-DC', 'UK00-UKNI-DC',
       'UKNI-IE00-AC', 'UKNI-UK00-DC'],
      dtype='object', length=212)

In [53]:
n.madd(
    "Generator",
    n.buses.index + " load-shedding",
    bus = n.buses.index,
    p_nom=1e5,
    marginal_cost = 10e3,
)

Index(['AL00 load-shedding', 'AT00 load-shedding', 'BA00 load-shedding',
       'BE00 load-shedding', 'BEOF load-shedding', 'BG00 load-shedding',
       'CH00 load-shedding', 'CY00 load-shedding', 'CZ00 load-shedding',
       'DE00 load-shedding', 'DEKF load-shedding', 'DKBH load-shedding',
       'DKE1 load-shedding', 'DKKF load-shedding', 'DKW1 load-shedding',
       'EE00 load-shedding', 'ES00 load-shedding', 'FI00 load-shedding',
       'FR00 load-shedding', 'GR00 load-shedding', 'GR03 load-shedding',
       'HR00 load-shedding', 'HU00 load-shedding', 'IE00 load-shedding',
       'ITCA load-shedding', 'ITCN load-shedding', 'ITCS load-shedding',
       'ITN1 load-shedding', 'ITS1 load-shedding', 'ITSA load-shedding',
       'ITSI load-shedding', 'ITVI load-shedding', 'LT00 load-shedding',
       'LUG1 load-shedding', 'LV00 load-shedding', 'ME00 load-shedding',
       'MK00 load-shedding', 'MT00 load-shedding', 'NL00 load-shedding',
       'NLLL load-shedding', 'NOM1 load-shedding', 

In [54]:
n.generators.loc["CY00 CCGT", "p_min_pu"] = 0 # remove minimum load of CCGT in Cyprus as this can exceed actual load.

In [55]:
dirname = os.path.dirname(save_path)  

if not os.path.exists(dirname):
    os.makedirs(dirname)

In [58]:
n.export_to_netcdf(save_path)

INFO:pypsa.io:Exported network cy1990_ty2029.nc has buses, loads, generators, storage_units, links


<xarray.Dataset>
Dimensions:                            (snapshots: 8760, investment_periods: 0,
                                        buses_i: 57, buses_t_p_i: 49,
                                        buses_t_marginal_price_i: 52,
                                        loads_i: 49, loads_t_p_set_i: 49,
                                        loads_t_p_i: 49, generators_i: 610,
                                        generators_t_p_max_pu_i: 170,
                                        ...
                                        storage_units_t_p_dispatch_i: 87,
                                        storage_units_t_p_store_i: 74,
                                        storage_units_t_state_of_charge_i: 89,
                                        storage_units_t_spill_i: 1,
                                        links_i: 212, links_t_p_max_pu_i: 212,
                                        links_t_p0_i: 149, links_t_p1_i: 149)
Coordinates: (12/22)
  * snapshots                          (snapshots) int64 0 1 2 ... 8758 8759
  * investment_periods                 (investment_periods) int64 
  * buses_i                            (buses_i) object 'AL00' 'AT00' ... 'UKNI'
  * buses_t_p_i                        (buses_t_p_i) object 'AL00' ... 'UKNI'
  * buses_t_marginal_price_i           (buses_t_marginal_price_i) object 'AL0...
  * loads_i                            (loads_i) object 'AL00' 'AT00' ... 'UKNI'
    ...                                 ...
  * storage_units_t_state_of_charge_i  (storage_units_t_state_of_charge_i) object ...
  * storage_units_t_spill_i            (storage_units_t_spill_i) object 'ES00...
  * links_i                            (links_i) object 'AL00-GR00-AC' ... 'U...
  * links_t_p_max_pu_i                 (links_t_p_max_pu_i) object 'AL00-GR00...
  * links_t_p0_i                       (links_t_p0_i) object 'AL00-ME00-AC' ....
  * links_t_p1_i                       (links_t_p1_i) object 'AL00-ME00-AC' ....
Data variables: (12/51)
    snapshots_snapshot                 (snapshots) datetime64[ns] 2010-01-01 ...
    snapshots_objective                (snapshots) float64 1.0 1.0 ... 1.0 1.0
    snapshots_stores                   (snapshots) float64 1.0 1.0 ... 1.0 1.0
    snapshots_generators               (snapshots) float64 1.0 1.0 ... 1.0 1.0
    investment_periods_objective       (investment_periods) object 
    investment_periods_years           (investment_periods) object 
    ...                                 ...
    links_p_nom                        (links_i) float64 400.0 400.0 ... 400.0
    links_carrier                      (links_i) object 'AC' 'AC' ... 'AC' 'DC'
    links_p_nom_opt                    (links_i) float64 400.0 400.0 ... 400.0
    links_t_p_max_pu                   (snapshots, links_t_p_max_pu_i) float64 ...
    links_t_p0                         (snapshots, links_t_p0_i) float64 69.4...
    links_t_p1                         (snapshots, links_t_p1_i) float64 -69....
Attributes:
    network__linearized_uc:  0
    network__multi_invest:   0
    network_name:            
    network_objective:       495741820.0144595
    network_pypsa_version:   0.25.2
    network_srid:            4326
    meta:                    {}

In [57]:
n.optimize(snapshots=n.snapshots[:24], solver_name="highs")

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 127.77it/s]
INFO:linopy.io: Writing time: 0.36s
INFO:linopy.solvers:Log file at /tmp/highs.log.


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d-dirty]
Copyright (c) 2023 HiGHS under MIT licence terms


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 26904 primals, 68840 duals
Objective: 4.96e+08
Solver model: available
Solver message: optimal



Presolving model
3384 rows, 25245 cols, 36198 nonzeros
3329 rows, 19002 cols, 25580 nonzeros
3191 rows, 18834 cols, 25274 nonzeros
Presolve : Reductions: rows 3191(-65649); columns 18834(-8070); elements 25274(-91437)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.6401293627e+08 Pr: 3119(1.06206e+07) 0s
       9509     4.9574182001e+08 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 9509
Objective value     :  4.9574182001e+08
HiGHS run time      :          0.29


/trinity/home/thomgeo/.conda/envs/pypsa-training/lib/python3.10/site-packages/linopy/common.py:124: UserWarning: Coordinates across variables not equal. Perform outer join.
  warn("Coordinates across variables not equal. Perform outer join.", UserWarning)
INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-fix-p-ramp_limit_up, Generator-fix-p-ramp_limit_down, Link-fix-p-lower, Link-fix-p-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


('ok', 'optimal')